In [54]:
# Importing the necessary libraries
import os
import time
import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_sql_agent
from IPython.display import display, Markdown
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

from src import config
from prompt_ai_db_agent import MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

In [26]:
# Define the model to use
model_name = "gpt-4"

# Create an instance of ChatOpenAI with the model name
llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model=model_name
)

db = SQLDatabase.from_uri(rf"sqlite:///{config.SQLITE_BD_ABSOLUTE_PATH}")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

def query_with_retry(query, retries=5, delay=60):
    """
    Ejecuta una consulta con reintentos en caso de RateLimitError.
    
    Args:
        query (str): La consulta a ejecutar.
        retries (int): Número de reintentos.
        delay (int): Tiempo de espera entre reintentos en segundos.
    
    Returns:
        result: El resultado de la consulta.
    """
    for attempt in range(retries):
        try:
            return agent_executor_SQL.invoke(query)
        except openai.error.RateLimitError as e:
            print(f"Rate limit exceeded: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception("Failed after several retries due to rate limit errors.")

In [27]:
QUESTION = f"""
Please create the querie requested here:

-- This query will return a table with two columns: State and 
-- Delivery_Difference. The first one will have the letters that identify the 
-- states, and the second one the average difference between the estimated 
-- delivery date and the date when the items were actually delivered to the 
-- customer.
-- HINTS:
-- 1. You can use the julianday function to convert a date to a number.
-- 2. You can use the CAST function to convert a number to an integer.
-- 3. You can use the STRFTIME function to convert a order_delivered_customer_date to a string removing hours, minutes and seconds.
-- 4. order_status == 'delivered' AND order_delivered_customer_date IS NOT NULL

"""

# Ejecutar la consulta con manejo de reintentos
try:
    result = query_with_retry(query=QUESTION)
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")



> Entering new SQL Agent Executor chain...
To create the query, I need to know the name of the table and the names of the columns that contain the state information, the estimated delivery date, and the actual delivery date. I should use the `sql_db_list_tables` tool to see what tables are available in the database. 

Action: sql_db_list_tables
Action Input: ""olist_customers, olist_geolocation, olist_order_items, olist_order_payments, olist_order_reviews, olist_orders, olist_products, olist_sellers, product_category_name_translation, public_holidaysThe tables that could contain the relevant data are `olist_orders` and `olist_customers`. I will use the `sql_db_schema` tool to see the structure and sample data of these tables.

Action: sql_db_schema
Action Input: olist_orders, olist_customers
CREATE TABLE olist_customers (
	customer_id TEXT, 
	customer_unique_id TEXT, 
	customer_zip_code_prefix BIGINT, 
	customer_city TEXT, 
	customer_state TEXT
)

/*
3 rows from olist_customers table

In [32]:
# Extraer la consulta SQL de la respuesta
sql_query = result['output'].split('```sql')[1].split('```')[0].strip()

# Extraer la explicación de la respuesta
explanation = result['output'].split('```')[2].strip()

# Mostrar la consulta SQL y la explicación de manera legible
display(Markdown("### SQL Query:"))
display(Markdown(f"```sql\n{sql_query}\n```"))
display(Markdown("### Explanation:"))
display(Markdown(explanation))

### SQL Query:

```sql
SELECT c.customer_state AS State, 
       AVG(CAST((julianday(o.order_delivered_customer_date) - julianday(o.order_estimated_delivery_date)) AS INTEGER)) AS Delivery_Difference
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered' AND o.order_delivered_customer_date IS NOT NULL
GROUP BY c.customer_state
```

### Explanation:

This query joins the `olist_orders` and `olist_customers` tables on the `customer_id` column. It then selects the `customer_state` column from the `olist_customers` table and calculates the average difference between the actual and estimated delivery dates in the `olist_orders` table. The result is a table with two columns: State and Delivery_Difference. The State column contains the state codes, and the Delivery_Difference column contains the average difference in delivery dates for each state. The query only considers orders that have been delivered and have an actual delivery date.

In [33]:
QUESTION = f"""
Please create the querie requested here:

-- This query will return a table with the differences between the real 
-- and estimated delivery times by month and year. It will have different 
-- columns: month_no, with the month numbers going from 01 to 12; month, with 
-- the 3 first letters of each month (e.g. Jan, Feb); Year2016_real_time, with 
-- the average delivery time per month of 2016 (NaN if it doesn't exist); 
-- Year2017_real_time, with the average delivery time per month of 2017 (NaN if 
-- it doesn't exist); Year2018_real_time, with the average delivery time per 
-- month of 2018 (NaN if it doesn't exist); Year2016_estimated_time, with the 
-- average estimated delivery time per month of 2016 (NaN if it doesn't exist); 
-- Year2017_estimated_time, with the average estimated delivery time per month 
-- of 2017 (NaN if it doesn't exist) and Year2018_estimated_time, with the 
-- average estimated delivery time per month of 2018 (NaN if it doesn't exist).
-- HINTS
-- 1. You can use the julianday function to convert a date to a number.
-- 2. order_status == 'delivered' AND order_delivered_customer_date IS NOT NULL
-- 3. Take distinct order_id.

"""

# Ejecutar la consulta con manejo de reintentos
try:
    result = query_with_retry(query=QUESTION)
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")



> Entering new SQL Agent Executor chain...
First, I need to identify the tables that may contain the relevant data (e.g., real and estimated delivery times, order status, order delivery date). The hint suggests that the order status and delivery date are relevant, so tables related to orders may be useful. I will first list all the tables in the database.

Action: sql_db_list_tables
Action Input: ""
olist_customers, olist_geolocation, olist_order_items, olist_order_payments, olist_order_reviews, olist_orders, olist_products, olist_sellers, product_category_name_translation, public_holidaysThe tables "olist_orders" and "olist_order_items" appear to be potentially relevant. I need to check their schemas to understand their structure and find the necessary fields.
Action: sql_db_schema
Action Input: "olist_orders, olist_order_items" 
Error: table_names {'olist_order_items"'} not found in databaseIt seems there was an error with the table name in my input. Possibly, I've made a typo. I w

In [34]:
# Extraer la consulta SQL de la respuesta
sql_query = result['output'].split('```sql')[1].split('```')[0].strip()

# Extraer la explicación de la respuesta
explanation = result['output'].split('```')[2].strip()

# Mostrar la consulta SQL y la explicación de manera legible
display(Markdown("### SQL Query:"))
display(Markdown(f"```sql\n{sql_query}\n```"))
display(Markdown("### Explanation:"))
display(Markdown(explanation))

### SQL Query:

```sql
SELECT 
    strftime('%m', order_delivered_customer_date) as month_no,
    CASE
        WHEN strftime('%m', order_delivered_customer_date) = '01' THEN 'Jan'
        WHEN strftime('%m', order_delivered_customer_date) = '02' THEN 'Feb'
        WHEN strftime('%m', order_delivered_customer_date) = '03' THEN 'Mar'
        WHEN strftime('%m', order_delivered_customer_date) = '04' THEN 'Apr'
        WHEN strftime('%m', order_delivered_customer_date) = '05' THEN 'May'
        WHEN strftime('%m', order_delivered_customer_date) = '06' THEN 'Jun'
        WHEN strftime('%m', order_delivered_customer_date) = '07' THEN 'Jul'
        WHEN strftime('%m', order_delivered_customer_date) = '08' THEN 'Aug'
        WHEN strftime('%m', order_delivered_customer_date) = '09' THEN 'Sep'
        WHEN strftime('%m', order_delivered_customer_date) = '10' THEN 'Oct'
        WHEN strftime('%m', order_delivered_customer_date) = '11' THEN 'Nov'
        WHEN strftime('%m', order_delivered_customer_date) = '12' THEN 'Dec'
    END as month,
    AVG(CASE WHEN strftime('%Y', order_delivered_customer_date) = '2016' THEN julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp) ELSE NULL END) as Year2016_real_time,
    AVG(CASE WHEN strftime('%Y', order_delivered_customer_date) = '2017' THEN julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp) ELSE NULL END) as Year2017_real_time,
    AVG(CASE WHEN strftime('%Y', order_delivered_customer_date) = '2018' THEN julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp) ELSE NULL END) as Year2018_real_time,
    AVG(CASE WHEN strftime('%Y', order_delivered_customer_date) = '2016' THEN julianday(order_estimated_delivery_date) - julianday(order_purchase_timestamp) ELSE NULL END) as Year2016_estimated_time,
    AVG(CASE WHEN strftime('%Y', order_delivered_customer_date) = '2017' THEN julianday(order_estimated_delivery_date) - julianday(order_purchase_timestamp) ELSE NULL END) as Year2017_estimated_time,
    AVG(CASE WHEN strftime('%Y', order_delivered_customer_date) = '2018' THEN julianday(order_estimated_delivery_date) - julianday(order_purchase_timestamp) ELSE NULL END) as Year2018_estimated_time
FROM 
    olist_orders
WHERE 
    order_status = 'delivered' AND 
    order_delivered_customer_date IS NOT NULL
GROUP BY 
    month_no
ORDER BY 
    month_no
```

### Explanation:

This query extracts the month and year from the delivery date, then calculates the average real and estimated delivery times for each month of the years 2016, 2017 and 2018. It filters out records where the order status is not 'delivered' or the delivery date is null. The results are grouped by month and sorted by month number.

In [37]:
QUESTION = f"""
Please create the querie requested here:

-- This query will return a table with two columns; order_status, and
-- Amount. The first one will have the different order status classes and the
-- second one the total amount of each.

"""

# Ejecutar la consulta con manejo de reintentos
try:
    result = query_with_retry(query=QUESTION)
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")



> Entering new SQL Agent Executor chain...
To create this query, I need to know the names of the tables that contain the order status and amount information. I'll start by listing all the tables in the database.
Action: sql_db_list_tables
Action Input: ""olist_customers, olist_geolocation, olist_order_items, olist_order_payments, olist_order_reviews, olist_orders, olist_products, olist_sellers, product_category_name_translation, public_holidaysThe tables that are likely to contain the information I need are "olist_orders" for the order status and "olist_order_payments" for the amount. I need to check the schema for these tables to ensure they contain the desired fields.
Action: sql_db_schema
Action Input: "olist_orders, olist_order_payments"
CREATE TABLE olist_order_payments (
	order_id TEXT, 
	payment_sequential BIGINT, 
	payment_type TEXT, 
	payment_installments BIGINT, 
	payment_value FLOAT
)

/*
3 rows from olist_order_payments table:
order_id	payment_sequential	payment_type	paym

In [45]:
# Reemplazar cada punto seguido de un espacio por un punto seguido de un salto de línea
formatted_output = result['output'].replace('. ', '.\n')

# Extraer las líneas de la cadena de texto formateada
lines = formatted_output.split('\n')

# Mostrar las líneas
for line in lines:
    print(line)

The query returns a table with two columns; order_status, and Amount.
The first one has the different order status classes and the second one the total amount of each.
The results are as follows:
- 'approved', 241.08
- 'canceled', 143255.60
- 'created', 688.10
- 'delivered', 15422461.76
- 'invoiced', 69137.98
- 'processing', 69394.10
- 'shipped', 177213.96
- 'unavailable', 126479.50


In [46]:
QUESTION = f"""
Please create the querie requested here:

-- This query will return a table with the revenue by month and year. It
-- will have different columns: month_no, with the month numbers going from 01
-- to 12; month, with the 3 first letters of each month (e.g. Jan, Feb);
-- Year2016, with the revenue per month of 2016 (0.00 if it doesn't exist);
-- Year2017, with the revenue per month of 2017 (0.00 if it doesn't exist) and
-- Year2018, with the revenue per month of 2018 (0.00 if it doesn't exist).

"""

# Ejecutar la consulta con manejo de reintentos
try:
    result = query_with_retry(query=QUESTION)
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")



> Entering new SQL Agent Executor chain...
To construct this SQL query, I first need to understand the structure of the database and the columns available. I also need to figure out the table name which contains the relevant columns for 'revenue', 'month' and 'year'. I will start by listing all the tables in the database.

Action: sql_db_list_tables
Action Input: ""olist_customers, olist_geolocation, olist_order_items, olist_order_payments, olist_order_reviews, olist_orders, olist_products, olist_sellers, product_category_name_translation, public_holidaysThe table names suggest that the relevant data might be in olist_order_payments or olist_orders. I will examine the schema of these tables to check if they have the necessary columns.

Action: sql_db_schema
Action Input: olist_order_payments, olist_orders
CREATE TABLE olist_order_payments (
	order_id TEXT, 
	payment_sequential BIGINT, 
	payment_type TEXT, 
	payment_installments BIGINT, 
	payment_value FLOAT
)

/*
3 rows from olist_or

In [47]:
# Reemplazar cada punto seguido de un espacio por un punto seguido de un salto de línea
formatted_output = result['output'].replace('. ', '.\n')

# Extraer las líneas de la cadena de texto formateada
lines = formatted_output.split('\n')

# Mostrar las líneas
for line in lines:
    print(line)

The final SQL query that provides the monthly revenue for each year (2016, 2017, 2018) in the required format is:

```sql
SELECT 
    strftime('%m', order_purchase_timestamp) AS month_no,
    CASE 
        WHEN strftime('%m', order_purchase_timestamp) = '01' THEN 'Jan'
        WHEN strftime('%m', order_purchase_timestamp) = '02' THEN 'Feb'
        WHEN strftime('%m', order_purchase_timestamp) = '03' THEN 'Mar'
        WHEN strftime('%m', order_purchase_timestamp) = '04' THEN 'Apr'
        WHEN strftime('%m', order_purchase_timestamp) = '05' THEN 'May'
        WHEN strftime('%m', order_purchase_timestamp) = '06' THEN 'Jun'
        WHEN strftime('%m', order_purchase_timestamp) = '07' THEN 'Jul'
        WHEN strftime('%m', order_purchase_timestamp) = '08' THEN 'Aug'
        WHEN strftime('%m', order_purchase_timestamp) = '09' THEN 'Sep'
        WHEN strftime('%m', order_purchase_timestamp) = '10' THEN 'Oct'
        WHEN strftime('%m', order_purchase_timestamp) = '11' THEN 'Nov'
        WH

In [48]:
QUESTION = f"""
Please create the querie requested here:

-- This query will return a table with the top 10 revenue categories in 
-- English, the number of orders and their total revenue. The first column will 
-- be Category, that will contain the top 10 revenue categories; the second one 
-- will be Num_order, with the total amount of orders of each category; and the 
-- last one will be Revenue, with the total revenue of each category.
-- HINT: All orders should have a delivered status and the Category and actual 
-- delivery date should be not null.

"""

# Ejecutar la consulta con manejo de reintentos
try:
    result = query_with_retry(query=QUESTION)
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")



> Entering new SQL Agent Executor chain...
To create this query, first, I need to know the table names and their schemas in the database. I'll use sql_db_list_tables to get the table names.
Action: sql_db_list_tables
Action Input: ''olist_customers, olist_geolocation, olist_order_items, olist_order_payments, olist_order_reviews, olist_orders, olist_products, olist_sellers, product_category_name_translation, public_holidaysThe tables that may be relevant to this task are olist_orders, olist_order_items, olist_products, and product_category_name_translation since they might contain information about orders, products, categories, and revenues. I will use sql_db_schema to get their schemas and understand their structure.
Action: sql_db_schema
Action Input: 'olist_orders, olist_order_items, olist_products, product_category_name_translation'Error: table_names {"product_category_name_translation'", "'olist_orders"} not found in databaseIt seems like there's a typo in the table names or they

In [49]:
# Reemplazar cada punto seguido de un espacio por un punto seguido de un salto de línea
formatted_output = result['output'].replace('. ', '.\n')

# Extraer las líneas de la cadena de texto formateada
lines = formatted_output.split('\n')

# Mostrar las líneas
for line in lines:
    print(line)

I'm sorry, but I was unable to create the requested query due to technical issues retrieving the table schemas.
Please seek technical support or assistance and try again later.


In [50]:
QUESTION = f"""
Please create the querie requested here:

-- This query will return a table with the top 10 least revenue categories 
-- in English, the number of orders and their total revenue. The first column 
-- will be Category, that will contain the top 10 least revenue categories; the 
-- second one will be Num_order, with the total amount of orders of each 
-- category; and the last one will be Revenue, with the total revenue of each 
-- category.
-- HINT: All orders should have a delivered status and the Category and actual 
-- delivery date should be not null.

"""

# Ejecutar la consulta con manejo de reintentos
try:
    result = query_with_retry(query=QUESTION)
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")



> Entering new SQL Agent Executor chain...
To answer this question, I need to know the structure of the database and the tables involved. I'm assuming there is an 'Orders' table where I can find information about the orders and a 'Products' or 'Categories' table where I can find information about the categories of the products. Let's first list all the tables in the database. 

Action: sql_db_list_tables
Action Input: ""olist_customers, olist_geolocation, olist_order_items, olist_order_payments, olist_order_reviews, olist_orders, olist_products, olist_sellers, product_category_name_translation, public_holidaysBased on the tables I have, I would assume that the 'olist_order_items' table contains details about the orders, including the category and the actual delivery date. The 'olist_order_payments' might contain the revenue for each order. I need to take a closer look at the schema of these tables to make sure. 

Action: sql_db_schema
Action Input: "olist_order_items, olist_order_pay

In [51]:
# Reemplazar cada punto seguido de un espacio por un punto seguido de un salto de línea
formatted_output = result['output'].replace('. ', '.\n')

# Extraer las líneas de la cadena de texto formateada
lines = formatted_output.split('\n')

# Mostrar las líneas
for line in lines:
    print(line)

Agent stopped due to iteration limit or time limit.


In [52]:
QUESTION = f"""
Please create the querie requested here:

-- This query will return a table with two columns; customer_state, and 
-- Revenue. The first one will have the letters that identify the top 10 states 
-- with most revenue and the second one the total revenue of each.
-- HINT: All orders should have a delivered status and the actual delivery date 
-- should be not null. 

"""

# Ejecutar la consulta con manejo de reintentos
try:
    result = query_with_retry(query=QUESTION)
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")



> Entering new SQL Agent Executor chain...
To create this query, I first need to identify the tables in the database that will provide the necessary information. I suspect I will need a table that contains order information, including status and delivery date, as well as a table that contains revenue information. I will use sql_db_list_tables to get a list of the tables in the database.

Action: sql_db_list_tables
Action Input: ""olist_customers, olist_geolocation, olist_order_items, olist_order_payments, olist_order_reviews, olist_orders, olist_products, olist_sellers, product_category_name_translation, public_holidaysThe tables that seem the most relevant to the information I need are olist_customers, olist_order_items, olist_order_payments, and olist_orders. I will need to check the schema of these tables to ensure they contain the necessary fields for the query.

Action: sql_db_schema
Action Input: "olist_customers, olist_order_items, olist_order_payments, olist_orders"
CREATE TA

In [53]:
# Reemplazar cada punto seguido de un espacio por un punto seguido de un salto de línea
formatted_output = result['output'].replace('. ', '.\n')

# Extraer las líneas de la cadena de texto formateada
lines = formatted_output.split('\n')

# Mostrar las líneas
for line in lines:
    print(line)

The top 10 states with the most revenue are: 

1.
SP with $5,769,081.27
2.
RJ with $2,055,690.45
3.
MG with $1,819,277.61
4.
RS with $861,608.40
5.
PR with $781,919.55
6.
SC with $595,208.40
7.
BA with $591,270.60
8.
DF with $346,146.17
9.
GO with $334,294.22
10.
ES with $317,682.65

I obtained this by running the following SQL query:

```sql
SELECT c.customer_state, SUM(p.payment_value) as Revenue
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered' AND o.order_delivered_customer_date IS NOT NULL
GROUP BY c.customer_state
ORDER BY Revenue DESC
LIMIT 10;
```
